In [1]:
source_dataset_url="graph_data/allmv_tmdb/allmv/graphsage/"
target_dataset_url="graph_data/allmv_tmdb/tmdb/graphsage/"
groundtruth_url="graph_data/allmv_tmdb/dictionaries/groundtruth"

In [4]:
source_dataset_URL="graph_data/douban/online/graphsage/"
target_dataset_URL="graph_data/douban/offline/graphsage/"
groundtruth="graph_data/douban/dictionaries/groundtruth"

In [1]:
acm_dataset_url = "graph_data/acm_dblp/acm/graphsage/"
dblp_dataset_url = "graph_data/acm_dblp/dblp/graphsage/"
groundtruth="graph_data/acm_dblp/dictionaries/groundtruth"

In [2]:
import utils.graph_utils as graph_utils
import json
import os
from scipy.io import loadmat
import numpy as np
import networkx as nx
from networkx.readwrite import json_graph
import torch
import torch.nn.functional as F
import random
import copy
from scipy.sparse import coo_matrix
import pandas as pd
from input.dataset import Dataset

/opt/data/private/interactive4388/bowen/.conda/envs/main/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
source_dataset = Dataset(source_dataset_URL)
target_dataset = Dataset(target_dataset_URL)

Dataset info:
- Nodes:  3906
- Edges:  8164
Dataset info:
- Nodes:  1118
- Edges:  1511


In [3]:
acm_dataset = Dataset(acm_dataset_url)
dblp_dataset = Dataset(dblp_dataset_url)

Dataset info:
- Nodes:  9872
- Edges:  39561
Dataset info:
- Nodes:  9916
- Edges:  44808


In [17]:
source_dataset.features

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
cur = torch.FloatTensor(acm_dataset.features)
F.normalize(cur)

tensor([[0.6356, 0.4376, 0.1667,  ..., 0.1459, 0.0104, 0.2292],
        [0.7368, 0.2456, 0.1637,  ..., 0.1501, 0.0000, 0.2183],
        [0.4805, 0.5428, 0.0801,  ..., 0.0890, 0.0000, 0.3470],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000]])

In [23]:
source_dataset.get_adjacency_matrix()

matrix([[0., 1., 1., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
source_dataset.get_edges()

array([[   0,    1],
       [   0,    2],
       [   1, 1272],
       ...,
       [3859, 3858],
       [3881, 3880],
       [3897, 3896]])

In [35]:
np.array(acm_dataset.get_adjacency_matrix())[0][:5]

array([0., 1., 1., 1., 1.])

In [37]:
acm_dataset.get_edges()[4]

array([0, 5])

In [17]:
from scipy.sparse import csr_matrix
def load_gt(path, id2idx_src=None, id2idx_trg=None, format='matrix'):    
    if id2idx_src:
        conversion_src = type(list(id2idx_src.keys())[0])
        conversion_trg = type(list(id2idx_trg.keys())[0])
    if format == 'matrix':
        # Dense
        """
        gt = np.zeros((len(id2idx_src.keys()), len(id2idx_trg.keys())))
        with open(path) as file:
            for line in file:
                src, trg = line.strip().split()                
                gt[id2idx_src[conversion_src(src)], id2idx_trg[conversion_trg(trg)]] = 1
        return gt
        """
        # Sparse
        row = []
        col = []
        val = []
        with open(path) as file:
            for line in file:
                src, trg = line.strip().split()
                row.append(id2idx_src[conversion_src(src)])
                col.append(id2idx_trg[conversion_trg(trg)])
                val.append(1)
        gt = csr_matrix((val, (row, col)), shape=(len(id2idx_src), len(id2idx_trg)))
    else:
        gt = {}
        with open(path) as file:
            for line in file:
                src, trg = line.strip().split()
                # print(src, trg)
                if id2idx_src:
                    gt[id2idx_src[conversion_src(src)]] = id2idx_trg[conversion_trg(trg)]
                else:
                    gt[int(src)] = int(trg)
    return gt
train_dict='graph_data/douban/dictionaries/node,split=0.1.train.dict'
train_dict = load_gt(train_dict, source_dataset.id2idx, target_dataset.id2idx, 'dict')

In [38]:
G2 = target_dataset.G
G2_edges = pd.DataFrame(G2.edges()).astype(int)


In [47]:
F.normalize(torch.tensor([[0.5,0.5],[0.5,0.5]]))

tensor([[0.7071, 0.7071],
        [0.7071, 0.7071]])

In [138]:
index = [1, 2, 3, 4]
columns = [5, 6, 7]

index_columns = pd.MultiIndex.from_product([index, columns], names=['index', 'columns'])
df = pd.DataFrame(index=index_columns).reset_index()

In [141]:
index_array, columns_array = np.meshgrid(index, columns)

In [ ]:
result = np.array([index_array.ravel(), columns_array.ravel()]).T
result[:]

In [152]:
sim_matrix = np.zeros((len(index) * len(columns), 3))
sim_matrix[:,0] = index_array.ravel()
sim_matrix[:,1] = columns_array.ravel()
sim_matrix[:,2] = index_array.ravel()+columns_array.ravel()

In [159]:
sim_matrix[[5,6],[1,2]]

array([6., 9.])

In [175]:
nodes1 = np.array([1, 2, 3])
nodes2 = np.array([4, 5, 6])
sims = np.array([0.5, 0.8, 0.2])

# 将 numpy 数组转换为 torch 张量
nodes1_tensor = torch.from_numpy(nodes1)
nodes2_tensor = torch.from_numpy(nodes2)
sims_tensor = torch.from_numpy(sims)

# 创建稀疏张量
indices = torch.stack([nodes1_tensor, nodes2_tensor])

In [4]:
setA = set('1')
setB = set('2')
setC = set('3')
set.union(setA,setB,setC)

{'1', '2', '3'}

In [156]:
dict_ = {4:5,6:7}
list(map(lambda x:dict_[x],np.array([4,6])))

[5, 7]

In [ ]:
def to_seed(x):
        return x
g1 = to_seed(list(df['index']))

In [140]:
def calculate_new(row):
    ep = 0.01
    #ACNs = len(list(nx.common_neighbors(G,row.name[0],row.name[1]+6))) + ep
    print(row['index'])
    #print(row.name[0],row.name[1]+6)
    ACNs = ep
    #Tver = ACNs**2 / (abs(len(setA) - len(setB)) + ep2)
    return ACNs**2
df['sim'] = df.apply(calculate_new,axis=1)

1
1
1
2
2
2
3
3
3
4
4
4


In [127]:
df

sim
index columns        
1     5        0.0001
      6        0.0001
      7        0.0001
2     5        0.0001
      6        0.0001
      7        0.0001
3     5        0.0001
      6        0.0001
      7        0.0001
4     5        0.0001
      6        0.0001
      7        0.0001

In [69]:
list(nx.common_neighbors(G2,'5','10'))

TypeError: object of type 'generator' has no len()

In [15]:
train_dict = 'graph_data/weibo_douban/dictionaries/node,split=0.1.train.dict'

part = train_dict.split('=')[1].split('.')

split_ratio = part[0]+'.' + part[1]

print(split_ratio)

0.1


In [104]:
num_rows = len(source_dataset.G.nodes())  # 获取行数，即字典中最大的键值加1
num_cols = len(target_dataset.G.nodes())  # 获取列数，即字典中最大的值加1
values = torch.ones(len(train_dict))  # 所有对应位置的值都设置为1
indices = torch.tensor([list(train_dict.keys()), list(train_dict.values())], dtype=torch.int64)
mat3 = torch.sparse_coo_tensor(indices, values, (num_rows, num_cols))
def Sparse_graph(A):
    if isinstance(A, np.matrix):
        A = torch.FloatTensor(A)
    indices = torch.nonzero(A).t()
    values = A[indices[0], indices[1]]
    A = torch.sparse.FloatTensor(indices, values, A.size())
    return A
mat1 = Sparse_graph(source_dataset.get_adjacency_matrix())
mat2 = Sparse_graph(target_dataset.get_adjacency_matrix())

In [12]:
A = target_dataset.get_adjacency_matrix()
if isinstance(A, np.matrix):
    A = torch.DoubleTensor(A)
indices = torch.nonzero(A).t()
#values = A[indices[0], indices[1]]
degree = torch.tensor(target_dataset.get_nodes_degrees())
degree = torch.pow(degree, -1)
degree[torch.isinf(degree)] = 0
values = A[indices[0], indices[1]] * degree[indices[0]]
A = torch.sparse_coo_tensor(indices, values, A.size(),dtype=torch.double)

In [17]:
A =A.double()

tensor(indices=tensor([[   0,    0,    0,  ..., 9511, 9512, 9513],
                       [   1,   21,   70,  ...,  120,  298,  205]]),
       values=tensor([0.0303, 0.0303, 0.0303,  ..., 1.0000, 1.0000, 1.0000]),
       size=(9514, 9514), nnz=329586, dtype=torch.float64,
       layout=torch.sparse_coo)

In [28]:
import torch.nn as nn
cur = nn.Embedding(len(target_dataset.G.nodes()),256).weight
torch.mm(A,cur.to(torch.float64))

tensor([[ 0.2251, -0.0734, -0.1275,  ...,  0.1526,  0.1652,  0.0676],
        [-0.0621,  0.1355,  0.1051,  ...,  0.1393, -0.0525, -0.0517],
        [ 0.2865, -0.4511, -0.0100,  ..., -0.5873, -0.0099, -0.0343],
        ...,
        [-0.0157, -1.7015,  0.5399,  ...,  0.4524,  0.0018, -0.5227],
        [-0.1711,  0.9633, -0.3868,  ...,  0.4405, -0.2844,  0.2109],
        [-0.1595,  1.3394, -0.2785,  ...,  0.5445, -0.2868, -0.9870]],
       dtype=torch.float64, grad_fn=<MmBackward0>)

In [ ]:
mat1 = Sparse_graph(source_dataset.get_adjacency_matrix(),source_dataset.get_nodes_degrees())
mat2 = Sparse_graph(target_dataset.get_adjacency_matrix())

In [112]:
top_half = torch.cat([mat1, mat3], dim=1)
bottom_half = torch.cat([mat2, mat3.t()], dim=1)
new_sparse_mat = torch.vstack([top_half, bottom_half])

In [174]:
node_source = torch.LongTensor(random.sample(list(list(range(200))),int(200 * 0.2)))

In [190]:
if isinstance(A, np.matrix):
        A = torch.FloatTensor(A)
    indices = torch.nonzero(A).t()
    #values = A[indices[0], indices[1]]
    degree = torch.sparse.sum(A, dim=1).to_dense()
    degree = torch.pow(degree, -1)
    degree[torch.isinf(degree)] = 0
    values = A[indices[0], indices[1]] * degree[indices[0]]
    A = torch.sparse.FloatTensor(indices, values, A.size())

TypeError: expected Tensor as element 0 in argument 0, but got dict_values

In [172]:
import pickle
data_dir = 'graph_data/weibo_douban/douban/graphsage/'
typea='s'
dense_feature_path = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(data_dir))),'profile_df.pkl')
if os.path.isfile(dense_feature_path):
    with open(dense_feature_path,'rb') as file:
        profile_df = pickle.load(file)
    sparse_feature_name = []
    dense_feature_name = []
    dense_f_list_transforms = {}
    profile_columns = ['username','description']
    print('loading dense features')
    for f in profile_columns:
        if f == 'description':
            continue
    ##如果特征值是一个列表，表示为密集特征，将列表中的元素合并为一个词汇表，并记录词汇表的长度
        if type(profile_df[f][0]) == list:
            dense_f_list = profile_df[f].values.tolist()
            vocab = []
            for sub_list in dense_f_list:
                for j in sub_list:
                    try:
                        vocab.append(j)
                    except:
                        print('empty feature')
                        continue
            vocab = list(set(vocab))
            vocab_len = len(vocab)
        """ dense_f_transform = []
        if typea == 's':
            dense_f_list = dense_f_list[:9514]
        else:
            dense_f_list = dense_f_list[9514:]
        for t in dense_f_list:
            dense_f_idx = torch.zeros(1, vocab_len).long()
            for w in t:
                idx = vocab.index(w)
                dense_f_idx[0, idx] = 1
            dense_f_transform.append(dense_f_idx)
        dense_f_list_transforms[f] = torch.cat(dense_f_transform, dim=0)
        dense_feature_name.append({'feature_name':f}) """

loading dense features


In [173]:
vocab_len

12693

In [152]:
dense_f_transform = []
if typea == 's':
    dense_f_list = dense_f_list[:9514]
else:
    dense_f_list = dense_f_list[9514:]
for t in dense_f_list:
    dense_f_idx = torch.zeros(1, vocab_len).long()
    for w in t:
        idx = vocab.index(w)
        dense_f_idx[0, idx] = 1
    dense_f_transform.append(dense_f_idx)
dense_f_list_transforms[f] = torch.cat(dense_f_transform, dim=0)
dense_feature_name.append({'feature_name':f})

In [164]:
profile_df.columns

Index(['username', 'gender', 'birthday', 'description', 'location'], dtype='object')

In [ ]:
import pickle
data_dir = 'graph_data/weibo_douban/douban/graphsage/'
typea='s'
dense_feature_path = os.path.join(os.path.dirname(os.path.dirname(data_dir)),'profile_df.pkl')
if os.path.isfile(dense_feature_path):
    with open(dense_feature_path,'rb') as file:
        profile_df = pickle.load(file)
    sparse_feature_name = []
    dense_feature_name = []
    dense_f_list_transforms = {}
    profile_columns = profile_df.columns
    print('loading dense features')
    for f in profile_columns:
    ##如果特征值是一个列表，表示为密集特征，将列表中的元素合并为一个词汇表，并记录词汇表的长度
        if type(profile_df[f][0]) == list:
            dense_f_list = profile_df[f].values.tolist()
            vocab = []
            for sub_list in dense_f_list:
                for j in sub_list:
                    try:
                        vocab.append(j)
                    except:
                        print('empty feature')
                        continue
            vocab = list(set(vocab))
            vocab_len = len(vocab)

            #然后，将每个列表转换为一个二进制的张量，其中每个位置表示是否存在对应的词。
            dense_f_transform = []
            if typea == 's':
                dense_f_list = dense_f_list[:len(source_dataset.G.nodes())]
            else:
                dense_f_list = dense_f_list[len(target_dataset.G.nodes()):]
            for t in dense_f_list:
                dense_f_idx = torch.zeros(1, vocab_len).long()
                for w in t:
                    idx = vocab.index(w)
                    dense_f_idx[0, idx] = 1
                dense_f_transform.append(dense_f_idx)
            dense_f_list_transforms[f] = torch.cat(dense_f_transform, dim=0)
            dense_feature_name.append({'feature_name':f})
            
        else:
            #如果特征值不是一个列表，表示为稀疏特征，使用LabelEncoder对其进行编码，并记录特征的维度。最后，将物品特征转换为torch.Tensor类型的矩阵。
            encoder = LabelEncoder()
            encoder.fit(profile_df[f])
            profile_df[f] = encoder.transform(profile_df[f])
            feature_dim = len(encoder.classes_)
            sparse_feature_name.append({'feature_name':f, 'feature_dim':feature_dim})
    sparse_feature_matrix = torch.from_numpy(profile_df[[f['feature_name'] for f in sparse_feature_name]].values)

In [140]:
values,indices = torch.sort(score,descending=True)
top1_indices = []
used_indices = []
score_masked = []
for i in range(values.shape[0]):
    j = 0
    index = indices[i][j]
    while index in used_indices:
        j = (j+1)%values.shape[1]
        index = indices[i][j]
    top1_indices.append(index)
    used_indices.append(index)
    score_masked.append(score[i][index])
top1_indices = torch.tensor(top1_indices)
score_masked = torch.tensor(score_masked)
top1_indices,score_masked

KeyboardInterrupt: 

In [ ]:
top_half = torch.sparse.cat([mat1, mat2], dim=1)
bottom_half = torch.sparse.cat([torch.zeros_like(padded_mat2), padded_mat2], dim=1)

In [ ]:


# 假设你有三个稀疏矩阵 mat1, mat2 和 mat3

# 将 mat1 和 mat2 分别作为一个大矩阵的左上角和右下角
num_rows = max(mat1.size(0), mat2.size(0))
num_cols = max(mat1.size(1), mat2.size(1))
padded_mat1 = torch.sparse_coo_tensor(mat1._indices(), mat1._values(), (num_rows, num_cols))
padded_mat2 = torch.sparse_coo_tensor(mat2._indices(), mat2._values(), (num_rows, num_cols))

# 将 mat3 和 mat3 的转置分别作为一个大矩阵的右上角和左下角
mat3_transposed = mat3.t()
num_rows = max(mat3.size(0), mat3_transposed.size(0))
num_cols = max(mat3.size(1), mat3_transposed.size(1))
padded_mat3 = torch.sparse_coo_tensor(mat3._indices(), mat3._values(), (num_rows, num_cols))
padded_mat3_transposed = torch.sparse_coo_tensor(mat3_transposed._indices(), mat3_transposed._values(), (num_rows, num_cols))

# 使用 cat 函数将左上角和右下角组合成一个大矩阵的上半部分
top_half = torch.sparse.cat([padded_mat1, torch.zeros_like(padded_mat1)], dim=1)
bottom_half = torch.sparse.cat([torch.zeros_like(padded_mat2), padded_mat2], dim=1)

# 使用 vstack 函数将上半部分和下半部分组合成一个新的稀疏大矩阵
new_sparse_mat = torch.sparse.vstack([top_half, bottom_half])

print(new_sparse_mat)

In [47]:
def my_collate_train(batch):
    #可能是s2t或者是t2s
    first_node = [item[0] for item in batch]
    pos_node = [item[1] for item in batch]
    neg_node = [item[2] for item in batch]
    print(first_node)

    first_node = torch.LongTensor(first_node)
    pos_node = torch.LongTensor(pos_node)
    neg_node = torch.LongTensor(neg_node)

    return [first_node, pos_node, neg_node]

In [42]:
import torch
from torch_geometric.nn import GINEConv

# 假设 edge_attr 是边的权重
edge_attr = torch.tensor([0.5, 0.8, 0.2, 0.4])  # 4 条边的权重
a = edge_attr.unsqueeze(-1)
a.shape

torch.Size([4, 1])

In [64]:
a = torch.randn(torch.Size([32,16]))
b = torch.randn(torch.Size([32,16]))
c = torch.sum(a*b,dim=1)

In [65]:
c

tensor([-2.3706, -5.5941, -3.3357,  4.2818,  5.8353, -2.3487, -8.4051, -7.4610,
         5.6333,  3.8732,  4.9585, -0.7757, -4.0768, -1.5527,  8.5214, -3.4207,
        -3.5349,  5.2682,  6.8391,  5.4776,  0.1327, -0.5032, -6.9506,  0.7837,
        -2.1135,  0.9928, -1.1655, 13.3179,  0.2021, -1.9950,  8.7415,  1.1583])

In [73]:
indices = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)
values = torch.tensor([1.0, 2.0, 3.0, 4.0], dtype=torch.float)
sparse_matrix = torch.sparse.FloatTensor(indices, values, torch.Size([3, 3]))
sparse_matrix.coalesce().indices()

tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])

In [48]:
from torch.utils.data import DataLoader
s2t_train_loader = DataLoader(s2t_train_set, shuffle=True, batch_size=128, collate_fn=my_collate_train)

In [49]:
a = iter(s2t_train_loader)

In [94]:
source_A_hat=None
s_model = torch.load('s_model.pth')
t_model = torch.load('t_model.pth')
""" s_embedding = torch.FloatTensor(s_model(source_A_hat,'s').cpu().detach().numpy())
t_embedding = torch.FloatTensor(t_model(source_A_hat,'t').cpu().detach().numpy())
torch.save(s_embedding, 's_embedding.pth') 
torch.save(t_embedding, 't_embedding.pth')  """    

" s_embedding = torch.FloatTensor(s_model(source_A_hat,'s').cpu().detach().numpy())\nt_embedding = torch.FloatTensor(t_model(source_A_hat,'t').cpu().detach().numpy())\ntorch.save(s_embedding, 's_embedding.pth') \ntorch.save(t_embedding, 't_embedding.pth')  "

In [77]:
mode = model.eval()
s2t_test_set = get_test_set(test_dict,'s2t')
s2t_test_loader = DataLoader(s2t_test_set, shuffle=True, batch_size=len(s2t_test_set), collate_fn=collate_two)
first_node,second_node = next(iter(s2t_test_loader))
s2t_source_outputs,s2t_target_outputs = model.two_forward(None,None,first_node,second_node,'s2t')
""" s2t_source_outputs = model(self.source_A_hat, 's')
s2t_target_outputs = model(self.target_A_hat, 't')  """
full_dict = load_gt(groundtruth, 'dict')
s2t_acc, s2t_S = get_acc(s2t_source_outputs, s2t_target_outputs, full_dict, alphas=[0,0,1], just_S=False)
print("Supervised s2t Acc: {}".format(s2t_acc))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/data/private/interactive63025/bowen/.conda/envs/bowen/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_505085/2786302525.py", line 5, in <module>
    s2t_source_outputs,s2t_target_outputs = model.two_forward(None,None,first_node,second_node,'s2t')
  File "/opt/data/private/interactive63025/bowen/Documents/MGLAlign-master/algorithms/MGLAlign/model.py", line 957, in two_forward
    #用拉普拉斯效果还行，但还是比不上GAlign的加上refine
  File "/opt/data/private/interactive63025/bowen/Documents/MGLAlign-master/algorithms/MGLAlign/model.py", line 1075, in two_link_predict
TypeError: encode() takes 2 positional arguments but 3 were given

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/data/private/interactive63025/bowen/.conda/envs/bowen/lib/python3.8/site-packages/IPython/core/intera

In [51]:
for index, (first_node, second_node, neg_node) in enumerate(s2t_train_loader):
    print(first_node)

[1069, 2044, 2901, 1698, 3725, 2250, 909, 2569, 1784, 2566, 1156, 3340, 2520, 3636, 2953, 871, 811, 583, 2378, 2583, 2032, 1843, 482, 3761, 529, 2687, 1453, 1223, 3556, 740, 251, 1528, 3296, 770, 3125, 1638, 1235, 670, 3297, 559, 1623, 3047, 232, 2298, 598, 2175, 2823, 2, 2430, 330, 1972, 203, 349, 3369, 1264, 3107, 2275, 2407, 2576, 1282, 2343, 1113, 2446, 240, 608, 563, 419, 289, 2932, 3123, 1272, 1415, 2991, 1593, 628, 1017, 3218, 1117, 2246, 1035, 689, 809, 1747, 3013, 3582, 704, 2625, 141, 1656, 3610, 1805, 2891, 3078, 1619, 2139, 1954, 1308, 2269, 2853, 1134, 3237, 3526, 1286, 3510, 1002, 2067, 1328, 3511, 472, 3351, 3533]
tensor([1069, 2044, 2901, 1698, 3725, 2250,  909, 2569, 1784, 2566, 1156, 3340,
        2520, 3636, 2953,  871,  811,  583, 2378, 2583, 2032, 1843,  482, 3761,
         529, 2687, 1453, 1223, 3556,  740,  251, 1528, 3296,  770, 3125, 1638,
        1235,  670, 3297,  559, 1623, 3047,  232, 2298,  598, 2175, 2823,    2,
        2430,  330, 1972,  203,  349, 3369,

In [ ]:
import pickle
attrs = pickle.load(open('../MAUIL-main/data/dblp/attrs','rb'))
attrs

In [20]:

a = min(source_dataset.get_nodes_degrees())
source_alpha = 1 + 3906/(np.log(source_dataset.get_nodes_degrees()).sum()-np.log(a)*3906)
b = np.power(source_dataset.get_nodes_degrees(),-source_alpha)
scaled_arr = np.interp(b, (b.min(), b.max()), (5, 50))


array([[16.24387701],
       [ 6.79447116],
       [ 5.04200278],
       ...,
       [50.        ],
       [50.        ],
       [50.        ]])

In [21]:
scaled_arr = np.interp(b, (b.min(), b.max()), (5, 50))
scaled_arr

array([16.24387701,  6.79447116,  5.04200278, ..., 50.        ,
       50.        , 50.        ])

In [61]:
path_len = 5
switch_prob=0.3
num_walks = 5
from collections import Counter
def add_edge(a, b,adj_dict):
    if a in adj_dict:
        neighbors = adj_dict[a]
    else:
        neighbors = set()
        adj_dict[a] = neighbors
    if b not in neighbors:
        neighbors.add(b)
    

def norm(counter):
    s = sum(counter.values())
    new_counter = Counter()
    for a, count in counter.items():
        new_counter[a] = counter[a] / s
    return new_counter

def compute_2hop_adj(adj):
    new_adj = np.zeros_like(adj)
    for i in range(len(new_adj)):
        new_adj[i] = adj[np.where(adj[i] == 1)[0].astype(int)].sum(axis=0)
    new_adj[new_adj>0] = 1
    return new_adj

def get_min_max_deg(adj, degree):
    deg_matrix = adj * degree
    max_deg = deg_matrix.max(axis=1)
    deg_matrix[deg_matrix == 0] = 1e10
    min_deg = deg_matrix.min(axis=1)
    return min_deg, max_deg

def get_dist_matrix():
        """
        Degree matrix
        """
        nes = len(source_dataset.G.edges())
        net = len(target_dataset.G.edges())
        nns = len(source_dataset.G.nodes())
        nnt = len(target_dataset.G.nodes())
        rectified_params_source = np.sqrt((net/nnt) / (nes/nns))
        rectified_params_target = 1 / rectified_params_source

        source_adj = source_dataset.get_adjacency_matrix()
        target_adj = target_dataset.get_adjacency_matrix()
        
        source_degree = source_adj.sum(axis=1) * rectified_params_source
        target_degree = target_adj.sum(axis=1) * rectified_params_target
        
        source_adj_2hop = compute_2hop_adj(source_adj)
        target_adj_2hop = compute_2hop_adj(target_adj)

        # we need to define max_deg_neib_source and min_deg_neib_source (target)
        min_deg_neib_source, max_deg_neib_source = get_min_max_deg(source_adj, source_degree)
        min_deg_neib_target, max_deg_neib_target = get_min_max_deg(target_adj, target_degree)

        min_deg_neib_source2, max_deg_neib_source2 = get_min_max_deg(source_adj_2hop, source_degree)
        min_deg_neib_target2, max_deg_neib_target2 = get_min_max_deg(target_adj_2hop, target_degree)
      
        dist = np.zeros((source_adj.shape[0], target_adj.shape[0]))

        for i in range(source_adj.shape[0]):
            
            """ dist[i] = np.abs(np.log(source_degree[i] + 1) - np.log(target_degree.reshape(1,1118) + 1)) * 2 + dist[i]
            dist[i] = np.abs(np.log(min_deg_neib_source[i] + 1) - np.log(min_deg_neib_target.reshape(1,1118) + 1)) + \
                        np.abs(np.log(max_deg_neib_source[i] + 1) - np.log(max_deg_neib_target.reshape(1,1118) + 1)) + dist[i]

            dist[i] = np.abs(np.log(min_deg_neib_source2[i] + 1) - np.log(min_deg_neib_target2.reshape(1,1118) + 1)) + \
                        np.abs(np.log(max_deg_neib_source2[i] + 1) - np.log(max_deg_neib_target2.reshape(1,1118) + 1)) + dist[i] """

            dist[i] = np.abs(np.log(source_degree[i] + 1) - np.log(target_degree.T + 1)) * 2 + dist[i]
            dist[i] = np.abs(np.log(min_deg_neib_source[i] + 1) - np.log(min_deg_neib_target.T + 1)) + \
                        np.abs(np.log(max_deg_neib_source[i] + 1) - np.log(max_deg_neib_target.T + 1)) + dist[i]

            dist[i] = np.abs(np.log(min_deg_neib_source2[i] + 1) - np.log(min_deg_neib_target2.T + 1)) + \
                        np.abs(np.log(max_deg_neib_source2[i] + 1) - np.log(max_deg_neib_target2.T + 1)) + dist[i]
        dist =  np.exp( - 5 * dist)
        return dist


def dist_to_distdict(dist):
    dist_dict = {}
    for node in source_dataset.G.nodes():
        for target_node in target_dataset.G.nodes():
            dist_dict[(source_dataset.id2idx[node], target_dataset.id2idx[target_node]+num_source_nodes)] = dist[source_dataset.id2idx[node], target_dataset.id2idx[target_node]]
    return dist_dict

In [18]:
for source in list(source_dataset.G.nodes())[:1]:
    hop = 2
    hop_num_list = []
    frontiers = {source}
    travel = [source]
    travel_set = {source}
    travel_hop = 1
    while travel_hop <= hop:
        nexts = set()
        node_size = 10
        for frontier in frontiers:
            frontier = int(frontier)
            if len(source_dataset.get_nodes_neighbors()[frontier]) > node_size:
                node_children = np.random.choice(source_dataset.get_nodes_neighbors()[frontier], node_size, replace=False)
            else:
                node_children = source_dataset.get_nodes_neighbors()[frontier]
            for current in node_children:
                if current not in travel_set:
                    travel.append(current)
                    nexts.add(current)
                    travel_set.add(current)
        frontiers = nexts
        hop_num_list.append(len(nexts))
        travel_hop += 1
    travel.remove(source)
    #feature_embedding = cal_embedding(travel, hop_num_list, p_lambda, deepwalk_embeddings)
hop_num_list

[2, 15]

In [1]:
source_dataset.get_nodes_degrees()

NameError: name 'source_dataset' is not defined

In [49]:
from utils.graph_utils import load_gt
num_source_nodes=3906

In [1]:
#得到的是索引
train_dict = load_gt("graph_data/douban/dictionaries/node,split=0.2.train.dict", source_dataset.id2idx, target_dataset.id2idx, 'dict')
""" for key,value in train_dict.items():
    train_dict[key] = value + num_source_nodes
inv_train_dict = {v:k for k, v in train_dict.items()} """

NameError: name 'load_gt' is not defined

In [87]:
source_adj_dict = {}
target_adj_dict = {}
for a, b in source_dataset.get_edges():
    add_edge(int(a), int(b),source_adj_dict)
    add_edge(int(b), int(a),source_adj_dict)

for c, d in target_dataset.get_edges():
    add_edge(int(c+num_source_nodes), int(d+num_source_nodes),target_adj_dict)
    add_edge(int(d+num_source_nodes), int(c+num_source_nodes),target_adj_dict)

source_adj_dict = {a: list(neighbors) for a, neighbors in source_adj_dict.items()}
target_adj_dict = {c: list(neighbors) for c, neighbors in target_adj_dict.items()}

adj_dict = [source_adj_dict,target_adj_dict]

dist_dict = dist_to_distdict(get_dist_matrix())

nets = [source_dataset.G, target_dataset.G]

# word_counter = Counter()
walk_counters = {}


In [ ]:
for net_index in range(2):
    for node in adj_dict[net_index]:
        curr_node = node
        curr_net = net_index
        #current_path_len = np.random.randint(1, path_len + 1)
        for _ in range(num_walks):
            for _ in range(path_len):
                if node in walk_counters:
                    walk_counter = walk_counters[node]
                else:
                    walk_counter = Counter()
                    walk_counters[node] = walk_counter

                if np.random.rand() > switch_prob:
                    neighbors_deg = [nets[curr_net].degree(str(ele) if curr_net==0 else str(ele-num_source_nodes)) for ele in adj_dict[curr_net][curr_node]]
                    next_node = np.random.choice(adj_dict[curr_net][curr_node], p=np.array(neighbors_deg)/np.sum(neighbors_deg))
                else:
                    # switch net now
                    # train_dict的target图索引要加上源图的结点数
                    if train_dict.get(curr_node) and curr_net==0:
                        next_node = train_dict.get(curr_node)
                        curr_net = 1 # switched
                    elif inv_train_dict.get(curr_node) and curr_net==1:
                        next_node = inv_train_dict.get(curr_node)
                        curr_net = 0 # switched
                    else:
                        cur_to_switch = (curr_net + 1) % 2
                        destination_nodes = adj_dict[cur_to_switch].keys()
                        if curr_net == 0:
                            p = np.array([dist_dict[(curr_node, ele)] for ele in destination_nodes])
                        elif curr_net == 1:
                            p = np.array([dist_dict[(ele, curr_node)] for ele in destination_nodes])
                        p /= p.sum()
                        next_node = np.random.choice(list(destination_nodes), p=p)
                        curr_net = cur_to_switch # switched
                walk_counter[next_node] += 1
                curr_node = next_node

normed_walk_counters = {a: norm(walk_counter) for a, walk_counter in walk_counters.items()}

prob_sums = Counter()

for a, normed_walk_counter in normed_walk_counters.items():
    for b, prob in normed_walk_counter.items():
        prob_sums[b] += prob

ppmis = {}

for a, normed_walk_counter in normed_walk_counters.items():
    for b, prob in normed_walk_counter.items():
        ppmi = np.log(prob / prob_sums[b] * len(prob_sums) / path_len)
        ppmis[(a, b)] = ppmi

In [ ]:
print("Random walk process")
pairs = []
nets = [self.Gs, self.Gt]
suffices = [0, len(self.Gs.nodes())]
id2idices = [self.id2idxs, self.id2idxt]

cur = 0

for net_index in range(2):
    for node in nets[net_index].nodes():
        for _ in range(self.num_walks):
            if nets[net_index].degree(node) == 0:
                continue
            curr_node = node
            cur = net_index
            for _ in range(self.walk_len):
                if np.random.rand() > self.switch_prob:
                    # choose by degree distribution
                    
                    neighbors = nets[cur].neighbors(curr_node)
                    list_neighbors= []
                    for neighbor in neighbors:
                        list_neighbors.append(neighbor)
                    neighbors_deg = [nets[cur].degree(ele) for ele in list_neighbors]
                    next_node = np.random.choice(list_neighbors, p=np.array(neighbors_deg)/np.sum(neighbors_deg))
                    curr_node = next_node
                else:
                    # switch net now
                    if self.test_dict.get(curr_node) and cur==0:
                        next_node = self.test_dict.get(curr_node)
                        cur = 1 # switched
                        curr_node = next_node
                    elif self.inv_test_dict.get(curr_node) and cur==1:
                        next_node = self.inv_test_dict.get(curr_node)
                        cur = 0 # switched
                        curr_node = next_node
                    else:
                        cur_to_switch = (cur + 1) % 2
                        destination_nodes = nets[cur_to_switch].nodes()
                        if cur == 0:
                            p = np.array([self.dist_dict[(curr_node, ele)] for ele in destination_nodes])
                        elif cur == 1:
                            p = np.array([self.dist_dict[(ele, curr_node)] for ele in destination_nodes])
                        p /= p.sum()
                        next_node = np.random.choice(destination_nodes, p=p)
                        cur = cur_to_switch # switched
                        curr_node = next_node
                if curr_node != node or cur != net_index:
                    pairs.append([id2idices[net_index][node] + suffices[net_index], id2idices[cur][curr_node] + suffices[cur]])
    cur = 1

In [231]:
def Laplacian_graph(A):
    for i in range(len(A)):
        A[i, i] = 1
    A = torch.FloatTensor(A)
    D_ = torch.diag(torch.sum(A, 0)**(-0.5))
    A_hat = torch.matmul(torch.matmul(D_,A),D_)
    A_hat = A_hat.float()
    indices = torch.nonzero(A_hat).t()
    values = A_hat[indices[0], indices[1]]
    A_hat = torch.sparse.FloatTensor(indices, values, A_hat.size())
    return A_hat, coo_matrix(A.detach().cpu().numpy())

In [239]:
source_feats = source_dataset.features
target_feats = target_dataset.features

In [70]:
def matrix2sparse(matrix):
    tensor = torch.Tensor(matrix)
    indices = torch.nonzero(tensor).t()
    values = tensor[indices[0], indices[1]]
    coo_tensor = torch.sparse.FloatTensor(indices, values, tensor.size())
    return coo_tensor
source_adj = matrix2sparse(source_dataset.get_adjacency_matrix())
target_adj = matrix2sparse(target_dataset.get_adjacency_matrix())

In [10]:
def load_gt(path, id2idx_src=None, id2idx_trg=None, format='matrix'):    
    if id2idx_src:
        conversion_src = type(list(id2idx_src.keys())[0])
        conversion_trg = type(list(id2idx_trg.keys())[0])
    if format == 'matrix':
        # Dense
        """
        gt = np.zeros((len(id2idx_src.keys()), len(id2idx_trg.keys())))
        with open(path) as file:
            for line in file:
                src, trg = line.strip().split()                
                gt[id2idx_src[conversion_src(src)], id2idx_trg[conversion_trg(trg)]] = 1
        return gt
        """
        # Sparse
        row = []
        col = []
        val = []
        with open(path) as file:
            for line in file:
                src, trg = line.strip().split()
                row.append(id2idx_src[conversion_src(src)])
                col.append(id2idx_trg[conversion_trg(trg)])
                val.append(1)
        gt = csr_matrix((val, (row, col)), shape=(len(id2idx_src), len(id2idx_trg)))
    else:
        gt = {}
        with open(path) as file:
            for line in file:
                src, trg = line.strip().split()
                # print(src, trg)
                if id2idx_src:
                    gt[id2idx_src[conversion_src(src)]] = id2idx_trg[conversion_trg(trg)]
                else:
                    gt[str(src)] = str(trg)
    return gt

In [11]:
groundtruth = load_gt("graph_data/douban/dictionaries/groundtruth", source_dataset.id2idx, target_dataset.id2idx, 'dict')

In [71]:
source_indices = torch.LongTensor(list(groundtruth.keys()))
target_indices = torch.LongTensor(list(groundtruth.values()))
source_adj.index_select(0,source_indices)
target_adj.index_select(0,target_indices)

tensor(indices=tensor([[   0,    0,    0,  ..., 1116, 1117, 1117],
                       [  45,  575,  834,  ..., 3902,  723, 1373]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(1118, 3906), nnz=7848, layout=torch.sparse_coo)

In [85]:
cross_adj = torch.sparse.mm(source_adj.index_select(0,source_indices).t(),target_adj.index_select(0,target_indices))

In [136]:
row = 4
col=4
rate=0.25
zeros_num = int(row* rate)
new_array = np.ones((row,col))
for i in range(row):
    indices = random.sample(range(col),zeros_num)
    new_array[i,indices] = 0
new_array

array([[1., 1., 0., 1.],
       [1., 1., 1., 0.],
       [1., 0., 1., 1.],
       [1., 1., 0., 1.]])

In [1]:
neg_list = []
for key,value in train_dict.items():
    neg_node = random.choice(list(range(num)))
    if type == 's2t':
        while neg_node == value:
            neg_node = random.choice(list(range(num)))
    else:
        while neg_node == key:
            neg_node = random.choice(list(range(num)))
    neg_list.append(neg_node)
if type == 's2t':
    pair = list(zip(list(train_dict.keys()), list(train_dict.values()),neg_list))
else:
    pair = list(zip(list(train_dict.values()), list(train_dict.keys()),neg_list))

NameError: name 'groundtruth' is not defined

In [ ]:
neg_list = []
for key,values in groundtruth.items():
    neg_node = random.choice(list(range(1118)))
    while neg_node == values:
        neg_node = random.choice(list(range(1118)))
    neg_list.append(neg_node)
neg_list

In [232]:
source_A_hat, _ = Laplacian_graph(source_dataset.get_adjacency_matrix())
source_A_mixup_hat,_ = Laplacian_graph(source_mixup_dataset.get_adjacency_matrix())

In [ ]:
for index,value in enumerate(source_dataset.get_nodes_neighbors()):
    print(index)

In [478]:
for element in source_dataset.get_nodes_neighbors()[0]:
    print(source_dataset.id2idx[element])

1
2


In [480]:
list(source_dataset.G.neighbors('0'))
source_dataset.id2idx[source_dataset.G.neighbors('0')[0]]

TypeError: 'dict_keyiterator' object is not subscriptable

In [240]:
source_ids = list(range(len(source_feats)))
target_ids = list(range(len(target_feats)))
np.random.shuffle(source_ids)
np.random.shuffle(target_ids)
shuffled_source_feats = source_feats[source_ids]
shuffled_target_feats = target_feats[target_ids]
row = source_A_hat.coalesce().indices()[0]
col = source_A_hat.coalesce().indices()[1]
new_indices = torch.stack([source_ids[row], source_ids[col]], dim=0)

In [271]:
row = source_A_hat.coalesce().indices()[0]
col = source_A_hat.coalesce().indices()[1]
new_indices = torch.stack([torch.tensor([source_ids[element.item()] for element in row]), torch.tensor([source_ids[element.item()] for element in col])], dim=0)
new_indices

tensor([[ 501,  501,  501,  ...,  646, 1558, 1558],
        [ 501, 3141, 3429,  ...,  646, 3685, 1558]])

In [266]:
torch.tensor([source_ids[element.item()] for element in row])

tensor([ 501,  501,  501,  ...,  646, 1558, 1558])

In [277]:
source_A_hat.coalesce().indices()[0]

tensor([   0,    0,    0,  ..., 3904, 3905, 3905])

In [281]:
shuffled_source_A_hat = source_A_hat.coalesce().clone()
source_row = source_A_hat.coalesce().indices()[0]
shuffled_source_A_hat.coalesce().indices()[0] = torch.tensor([source_ids[element.item()] for element in source_row])
source_col = source_A_hat.coalesce().indices()[1]
shuffled_source_A_hat.coalesce().indices()[1] = torch.tensor([source_ids[element.item()] for element in source_col])

In [556]:
import pandas as pd
results = pd.read_csv('0_experiment_results.csv')
mean = results.mean()
results = results.append(mean,ignore_index=True).rename(index={len(results): 'mean_values'})
results.round(4)

/tmp/ipykernel_1731490/5846151.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(mean,ignore_index=True).rename(index={len(results): 'mean_values'})


,Accuracy,MAP,AUC,Precision_1,Precision_5,Precision_10
0,0.4284,0.5381,0.9904,0.4240,0.6807,0.7701
1,0.4320,0.5415,0.9906,0.4293,0.6780,0.7737
2,0.4284,0.5423,0.9904,0.4267,0.6771,0.7818
3,0.4258,0.5345,0.9899,0.4186,0.6789,0.7639
4,0.4374,0.5432,0.9905,0.4293,0.6807,0.7728
mean_values,0.4304,0.5399,0.9904,0.4256,0.6791,0.7725


In [533]:
results = pd.read_csv('1_experiment_results.csv')
mean = results.mean()
results = results.append(mean,ignore_index=True).rename(index={len(results): 'mean_values'})
results.round(4)

/tmp/ipykernel_1731490/1415775847.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(mean,ignore_index=True).rename(index={len(results): 'mean_values'})


,Accuracy,MAP,AUC,Precision_1,Precision_5,Precision_10
0,0.4329,0.5408,0.9908,0.4249,0.6673,0.7692
1,0.4267,0.5323,0.9899,0.4275,0.6610,0.7701
2,0.4338,0.5370,0.9904,0.4302,0.6762,0.7800
3,0.4374,0.5457,0.9907,0.4365,0.6762,0.7665
4,0.4347,0.5460,0.9901,0.4320,0.6789,0.7683
mean_values,0.4331,0.5403,0.9904,0.4302,0.6719,0.7708


In [157]:
def gen_degrees(neibor_dict):
        degrees = {}
        for key,value in neibor_dict.items():
            degrees[key] = len(value)
        return degrees
degrees1 = gen_degrees(neibor_dict1)

In [104]:
nodes_file = "graph_data/douban/online/shuffle/graphsage/G.json"
G_data = json.load(open(nodes_file))
G = json_graph.node_link_graph(G_data)

all_nodes = np.array(G.nodes())
all_edges = G.edges()

In [126]:
feature_file = 'graph_data/douban/online/shuffle/graphsage/feats.npy'
all_features = None
if os.path.isfile(feature_file):
    all_features = np.load(feature_file)

In [71]:
lam = np.random.beta(4.0, 4.0)

In [ ]:
mixup_node = {}
for key,value in neibor_dict1.items():
    max_node = list(value)[0]
    for index,node in enumerate(value):
        if degrees1[node] > degrees1[max_node]:
            max_node = node
    mixup_node[key] = max_node
mixup_node

In [217]:
random.sample(mixup_node,5)

TypeError: Population must be a sequence or set.  For dicts, use list(d).

In [176]:
mixup_features = np.zeros(all_features.shape)
mixup_neighbor = {}

In [167]:
for key,value in mixup_node.items():
    mixup_features[key] = all_features[key]*lam + (1-lam)*all_features[value]
    #mixup_degrees[key] = round(degrees1[key]*lam + (1-lam)*degrees1[value])
    #TODO:对于边这种离散的，到底怎么mixup，对度数mixup以后该如何选择保留和增加哪些边，如果通过训练自适应应该怎么弄，先试试随机
    #有两种思路，一种是在原本的图基础上对满足要求的节点mixup节点属性和度数即边，这样节点总数不变；另一种是生成新的节点，原本的节点要保留不变，节点总数变

In [206]:
neighbor_dict = copy.deepcopy(neibor_dict1)
for key,value in mixup_node.items():
    mixup_neighbor[key] = set(random.sample(neighbor_dict[key]-{value}, round(degrees1[key]*lam)-1)) | set(random.sample(neighbor_dict[value]-{key}, round(degrees1[value]*(1-lam)))) | {value}

In [212]:
for key,value in mixup_neighbor.items():
    if key==0:
        for index,neighbor in enumerate(value):
            print(neighbor)

320
1
132
196
969
973
666
1243
862
95
1406
421
1261
367
369
2229
182
760
2041
382


In [202]:
a = neighbor_dict[0]-{666}
a

{1, 434, 925, 969, 1243, 2041, 2065, 2229}

In [203]:
neighbor_dict[0]

{1, 434, 666, 925, 969, 1243, 2041, 2065, 2229}

In [138]:
import torch
DIFF_S = torch.load('DIFFAlign_smi_normalize.pt').detach().cpu()
DL_S = torch.load('DeepLink_smi.pt').detach().cpu()

In [140]:
max_indices_DIFF = torch.argmax(DIFF_S,dim=1)
max_indices_DIFF

tensor([211,  46, 211,  ..., 211, 211, 211])

In [141]:
max_indices_DEEP = torch.argmax(DL_S,dim=1)
max_indices_DEEP

tensor([568, 550, 299,  ..., 940, 226, 848])

In [123]:
(DIFF_S-DL_S).sum()

tensor(-4748659.5000)

In [122]:
DL_S.shape

torch.Size([3906, 1118])

In [70]:
DIFF_S[0],DL_S[0]

(tensor([ 0.6187,  0.7752,  0.9923,  ...,  0.4665, -0.1100,  0.0605]),
 tensor([0.9858, 0.7646, 1.0632,  ..., 1.4942, 0.7939, 1.3230]))

In [136]:
source_embedding_DIFF = torch.load('DIFFAlign_data.pt')[0].detach().cpu()
target_embedding_DIFF = torch.load('DIFFAlign_data.pt')[1].detach().cpu()

In [116]:
source_embedding_DEEP = torch.load('DeepLink_data.pt')[0].detach().cpu()
target_embedding_DEEP = torch.load('DeepLink_data.pt')[1].detach().cpu()

In [119]:
(source_embedding_DIFF-source_embedding_DEEP).sum()

tensor(3400.8481)

In [ ]:
acc, MAP, top5, top10, AUC,top1 = get_statistics(S, groundtruth, use_greedy_match=False, get_all_metric=True)

In [28]:
target_dataset.get_edges()[2]

array([0, 3])

In [33]:
def node_link_data(G, attrs=None):
    """Returns data in node-link format that is suitable for JSON serialization
    and use in Javascript documents.

    Parameters
    ----------
    G : NetworkX graph

    attrs : dict
        A dictionary that contains five keys 'source', 'target', 'name',
        'key' and 'link'.  The corresponding values provide the attribute
        names for storing NetworkX-internal graph data.  The values should
        be unique.  Default value::

            dict(source='source', target='target', name='id',
                 key='key', link='links')

        If some user-defined graph data use these attribute names as data keys,
        they may be silently dropped.

    Returns
    -------
    data : dict
       A dictionary with node-link formatted data.

    Raises
    ------
    NetworkXError
        If values in attrs are not unique.

    Examples
    --------
    >>> from networkx.readwrite import json_graph
    >>> G = nx.Graph([("A", "B")])
    >>> data1 = json_graph.node_link_data(G)
    >>> H = nx.gn_graph(2)
    >>> data2 = json_graph.node_link_data(
    ...     H, {"link": "edges", "source": "from", "target": "to"}
    ... )

    To serialize with json

    >>> import json
    >>> s1 = json.dumps(data1)
    >>> s2 = json.dumps(
    ...     data2, default={"link": "edges", "source": "from", "target": "to"}
    ... )

    Notes
    -----
    Graph, node, and link attributes are stored in this format.  Note that
    attribute keys will be converted to strings in order to comply with JSON.

    Attribute 'key' is only used for multigraphs.

    See Also
    --------
    node_link_graph, adjacency_data, tree_data
    """
    multigraph = G.is_multigraph()
    # Allow 'attrs' to keep default values.
    if attrs is None:
        attrs = _attrs
    else:
        attrs.update({k: v for (k, v) in _attrs.items() if k not in attrs})
    name = attrs["name"]
    source = attrs["source"]
    target = attrs["target"]
    links = attrs["link"]
    # Allow 'key' to be omitted from attrs if the graph is not a multigraph.
    key = None if not multigraph else attrs["key"]
    if len({source, target, key}) < 3:
        raise nx.NetworkXError("Attribute names are not unique.")
    data = {
        "directed": G.is_directed(),
        "multigraph": multigraph,
        "graph": G.graph,
        "nodes": [dict(chain(G.nodes[n].items(), [(name, n)])) for n in G],
    }
    if multigraph:
        data[links] = [
            dict(chain(d.items(), [(source, u), (target, v), (key, k)]))
            for u, v, k, d in G.edges(keys=True, data=True)
        ]
    else:
        data[links] = [
            dict(chain(d.items(), [(source, u), (target, v)]))
            for u, v, d in G.edges(data=True)
        ]
    return data

In [153]:
source_dataset

In [150]:
conversion_src = type(source_dataset.G.nodes['0']) 
conversion_src

dict

In [112]:
A1 = source_dataset.get_adjacency_matrix()
A2 = target_dataset.get_adjacency_matrix()

In [113]:
from numpy import inf, nan
n1 = A1.shape[0]
n2 = A2.shape[0]
d1 = 1 / A1.sum(axis=1)
d2 = 1 / A2.sum(axis=1)
d1[d1 == inf] = 0
d2[d2 == inf] = 0
d1 = d1.reshape(-1,1)*np.ones(d1.T.shape)
d2 = d2.reshape(-1,1)*np.ones(d2.T.shape)

In [130]:
from numpy import inf, nan
n1 = A1.shape[0]
n2 = A2.shape[0]
d1 = 1 / A1.sum(axis=1)
d2 = 1 / A2.sum(axis=1)
d1[d1 == inf] = 0
d2[d2 == inf] = 0
d1 = np.array(d1.reshape(-1,1))
d2 = np.array(d2.reshape(-1,1))

In [135]:
from scipy.io import loadmat
dict_H = loadmat('graph_data/douban/H.mat')
dict_H

{'__header__': b'MATLAB 5.0 MAT-file Platform: nt, Created on: Mon Nov 19 14:36:11 2018',
 '__version__': '1.0',
 '__globals__': [],
 'H': array([[0.28942624, 0.        , 0.2532979 , ..., 0.        , 0.        ,
         0.19280219],
        [0.        , 0.        , 0.        , ..., 0.07902424, 0.        ,
         0.        ],
        [0.18542083, 0.        , 0.38791765, ..., 0.14129021, 0.        ,
         0.13306451],
        ...,
        [0.        , 0.        , 0.        , ..., 0.1961202 , 0.        ,
         0.22020819],
        [0.        , 0.        , 0.        , ..., 0.15542971, 0.        ,
         0.15794968],
        [0.31150195, 0.61817088, 0.23167164, ..., 0.        , 0.        ,
         0.19244519]])}

In [61]:
G1 = source_dataset.G
G2 = target_dataset.G
res1 = json_graph.node_link_data(G1)
res2 = json_graph.node_link_data(G2)
id2idx1 = source_dataset.id2idx
id2idx2 = target_dataset.id2idx

new_nodes_idxs1 = np.arange(len(G1.nodes()))
new_nodes_idxs2 = np.arange(len(G1.nodes()), len(G1.nodes()) + len(G2.nodes()))

new_nodes = []
for idx, node in enumerate(res1["nodes"]):
    original_index = id2idx1[node["id"]]
    node["id"] = str(original_index)
    new_nodes.append(node)
for idx, node in enumerate(res2["nodes"]):
    original_index = id2idx2[node["id"]]
    node["id"] = str(int(original_index) + len(G1.nodes()))
    new_nodes.append(node)

new_id2idx = {}
for node in new_nodes:
    new_id2idx[node["id"]] = int(node["id"])

new_links = []
for link in res1["links"]:
    new_source_index = str(id2idx1[link["source"]])
    new_target_index = str(id2idx1[link["target"]])
    new_links.append({
        'source': new_source_index,
        'target': new_target_index
    })
for link in res2["links"]:
    new_source_index = str(int(id2idx2[link["source"]]) + len(G1.nodes()))
    new_target_index = str(int(id2idx2[link["target"]]) + len(G1.nodes()))
    if int(new_source_index) > 11724 or int(new_target_index) >11724:
        print('yes')
    new_links.append({
        'source': new_source_index,
        'target': new_target_index
    })


In [62]:
new_features = None
features1 = source_dataset.features
features2 = target_dataset.features

if features1 is not None and features2 is not None:
    if features1.shape[1] != features2.shape[1]:
        print("Can not create new features due to different features shape.")
    new_features = np.zeros((features1.shape[0] + features2.shape[0], features1.shape[1]))
    for i, feat in enumerate(features1):
        new_features[i] = feat
    for i, feat in enumerate(features2):
        new_features[i+len(G1.nodes())] = feat

new_res = json_graph.node_link_data(G1)
new_res["nodes"] = new_nodes
new_res["links"] = new_links

G = json_graph.node_link_graph(new_res)  

In [200]:
G1.nodes()

NodeView(('2', '25', '166', '265', '274', '347', '458', '481', '509', '537', '672', '771', '917', '930', '1137', '1199', '1207', '1282', '1338', '1464', '1545', '1737', '1760', '1809', '2978', '3301', '3363', '3479', '3490', '3542', '3616', '3742', '3833', '3844', '3912', '4285', '4386', '4441', '4564', '4596', '4741', '4884', '4930', '5040', '5140', '5591', '5631', '5671', '5696', '6156', '6225', '6286', '6378', '3', '290', '863', '1586', '2041', '2043', '2326', '2592', '2600', '2948', '2962', '3074', '3085', '3099', '3116', '3121', '3418', '3480', '3622', '3658', '3681', '4189', '4307', '4316', '4385', '4464', '4843', '4854', '5009', '5461', '5704', '5758', '6395', '4', '11', '293', '311', '380', '395', '443', '475', '496', '668', '737', '800', '843', '918', '992', '1058', '1068', '1076', '1139', '1281', '1371', '1711', '1716', '2124', '2258', '2262', '2431', '2439', '2596', '2800', '2807', '2944', '2968', '3080', '3288', '3440', '3530', '3587', '3694', '3919', '4245', '4408', '4474'

In [31]:
node_to_check = '2'

if node_to_check in G2.nodes:
    print(f"节点 {node_to_check} 存在于图中。")
else:
    print(f"节点 {node_to_check} 不存在于图中。")

节点 2 存在于图中。


In [ ]:
# 遍历边
for edge in G2.edges:
    source_node, target_node = edge
    if source_node == node_to_check or target_node == node_to_check:
        print(f"节点 {node_to_check} 存在于边 {edge}")

In [8]:
def has_duplicate_keys(dictionary):
    seen = set()
    for key in dictionary:
        if key in seen:
            return True
        seen.add(key)
    return False

result = has_duplicate_keys(id2idx2)

if result:
    print("字典中存在重复的键")
else:
    print("字典中的键都是唯一的")

字典中的键都是唯一的


In [18]:
source_feats = source_dataset.features
target_feats = target_dataset.features

if source_feats is None:
    source_feats = np.zeros((len(source_dataset.G.nodes()), 1))
    target_feats = np.zeros((len(target_dataset.G.nodes()), 1))

for i in range(len(source_feats)):
    if source_feats[i].sum() == 0:
        source_feats[i, -1] = 1
for i in range(len(target_feats)):
    if target_feats[i].sum() == 0:
        target_feats[i, -1] = 1
if source_feats is not None:
    source_feats = torch.FloatTensor(source_feats)
    target_feats = torch.FloatTensor(target_feats)
    source_feats = source_feats.cuda()
    target_feats = target_feats.cuda()
source_feats = F.normalize(source_feats)
target_feats = F.normalize(target_feats)

In [21]:
groundtruth_gt = graph_utils.load_gt(groundtruth,format='dict')

In [ ]:
source_gt = list(groundtruth_gt.keys())
target_gt = list(groundtruth_gt.values())

In [28]:
in_degrees = dict(source_dataset.G.in_degree(source_gt))
out_degrees = dict(source_dataset.G.out_degree(source_gt))

AttributeError: 'Graph' object has no attribute 'in_degree'

In [13]:
source_A_hat, _ = Laplacian_graph(source_dataset.get_adjacency_matrix())
target_A_hat, _ = Laplacian_graph(target_dataset.get_adjacency_matrix())
source_feats = source_dataset.features
target_feats = target_dataset.features

In [18]:
np.arange(3906)

array([   0,    1,    2, ..., 3903, 3904, 3905])

torch.Tensor

In [133]:
class GCN(nn.Module):
    """
    The GCN multistates block
    """
    def __init__(self, activate_function, input_dim, output_dim):
        """
        activate_function: Tanh
        input_dim: input features dimensions
        output_dim: output features dimensions
        """
        super(GCN, self).__init__()
        if activate_function is not None:
            self.activate_function = get_act_function(activate_function)
        else:
            self.activate_function = None
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.linear = nn.Linear(input_dim, output_dim, bias=False)
        init_weight(self.modules(), activate_function)
    
    def forward(self, input, A_hat):
        output = self.linear(input)
        output = torch.matmul(A_hat, output)
        if self.activate_function is not None:
            output = self.activate_function(output)
        return output

def get_act_function(activate_function):
    """
    Get activation function by name
    :param activation_fuction: Name of activation function 
    """
    if activate_function == 'sigmoid':
        activate_function = nn.Sigmoid()
    elif activate_function == 'relu':
        activate_function = nn.ReLU()
    elif activate_function == 'tanh':
        activate_function = nn.Tanh()
    else:
        return None
    return activate_function

def init_weight(modules, activation):
    """
    Weight initialization
    :param modules: Iterable of modules
    :param activation: Activation function.
    """
    for m in modules:
        if isinstance(m, nn.Linear):
            if activation is None:
                m.weight.data = init.xavier_uniform_(m.weight.data) #, gain=nn.init.calculate_gain(activation.lower()))
            else:
                m.weight.data = init.xavier_uniform_(m.weight.data, gain=nn.init.calculate_gain(activation.lower()))
            if m.bias is not None:
                m.bias.data = init.constant_(m.bias.data, 0.0)

In [134]:
GCNs=[]
activate_function="Tanh"
input_dim = 538
output_dim = 200
GCNs.append(GCN(activate_function, input_dim, output_dim))
input_dim = GCNs[-1].output_dim
GCNs = nn.ModuleList(GCNs)

In [ ]:
def get_edges(G, id2idx):
    edges1 = [(id2idx[n1], id2idx[n2]) for n1, n2 in G.edges()]
    edges2 = [(id2idx[n2], id2idx[n1]) for n1, n2 in G.edges()]
    
    edges = edges1 + edges2
    edges = np.array(edges)
    return edges

In [135]:
def graph_augmentation(self, dataset, type_aug='remove_edges'):
        """
        Generate small noisy graph from original graph
        :params dataset: original graph
        :params type_aug: type of noise added for generating new graph
        """
        edges = dataset.get_edges()
        adj = dataset.get_adjacency_matrix()
        
        if type_aug == "remove_edges":
            num_edges = len(edges)
            num_remove = int(len(edges) * self.args.noise_level)
            index_to_remove = np.random.choice(np.arange(num_edges), num_remove, replace=False)
            edges_to_remove = edges[index_to_remove]
            for i in range(len(edges_to_remove)):
                adj[edges_to_remove[i, 0], edges_to_remove[i, 1]] = 0
                adj[edges_to_remove[i, 1], edges_to_remove[i, 0]] = 0
        elif type_aug == "add_edges":
            num_edges = len(edges)
            num_add = int(len(edges) * self.args.noise_level)
            count_add = 0
            while count_add < num_add:
                random_index = np.random.randint(0, adj.shape[1], 2)
                if adj[random_index[0], random_index[1]] == 0:
                    adj[random_index[0], random_index[1]] = 1
                    adj[random_index[1], random_index[0]] = 1
                    count_add += 1
        elif type_aug == "change_feats":
            feats = np.copy(dataset.features)
            num_nodes = adj.shape[0]
            num_nodes_change_feats = int(num_nodes * self.args.noise_level)
            node_to_change_feats = np.random.choice(np.arange(0, adj.shape[0]), num_nodes_change_feats, replace=False)
            for node in node_to_change_feats:
                feat_node = feats[node]
                feat_node[feat_node == 1] = 0
                feat_node[np.random.randint(0, feats.shape[1], 1)[0]] = 1
            feats = torch.FloatTensor(feats)
            if self.args.cuda:
                feats = feats.cuda()
            return feats
        new_adj_H, _ = Laplacian_graph(adj)
        if self.args.cuda:
            new_adj_H = new_adj_H.cuda()
        return new_adj_H